In [13]:
import sys
!{sys.executable} -m pip install -r requeriments.txt


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [15]:
data = pd.read_csv('data/customers.csv')

In [16]:
def process_dataframe(data):
    num_data = data.select_dtypes(include = [np.number])
    num_data = num_data.drop(columns=['PipedriveId', 'Unnamed: 0'])
    return num_data

In [17]:
num_data = process_dataframe(data)
num_data.isnull().values.any()

False

In [18]:
X = num_data.drop(columns='Churned')
y = num_data.Churned

In [19]:
X.head()

,NumberOfDaysActive,AmountTasksPerMonth,AmountTasksLastMonth,FirstPlanVariantId_5ee8e2ac6237070d18ade4ee,FirstPlanVariantId_5f8e8ea23a5ed4dc29b70522,FirstPlanVariantId_611648ffef7ea3c81029de0e
0,58,0.00000,0,0,1,0
1,380,0.00000,0,0,1,0
2,402,0.00000,0,0,1,0
3,402,0.00000,0,0,1,0
4,122,1.47541,0,1,0,0


In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 6 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   NumberOfDaysActive                           1432 non-null   int64  
 1   AmountTasksPerMonth                          1432 non-null   float64
 2   AmountTasksLastMonth                         1432 non-null   int64  
 3   FirstPlanVariantId_5ee8e2ac6237070d18ade4ee  1432 non-null   int64  
 4   FirstPlanVariantId_5f8e8ea23a5ed4dc29b70522  1432 non-null   int64  
 5   FirstPlanVariantId_611648ffef7ea3c81029de0e  1432 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 67.2 KB


In [21]:
import mlflow

In [22]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("churn-prediction-experiment")

<Experiment: artifact_location='./mlruns/1', creation_time=1669824350997, experiment_id='1', last_update_time=1669824350997, lifecycle_stage='active', name='churn-prediction-experiment', tags={}>

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.lognormal('SVM_C', 0, 1.0),
        'kernel': hp.choice('kernel', ['linear', 'rbf'])
    },
    {
        'type': 'rf',
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'criterion': hp.choice('criterion', ['gini', 'entropy'])
    },
    {
        'type': 'logreg',
        'C': hp.lognormal('LR_C', 0, 1.0),
        'solver': hp.choice('solver', ['liblinear', 'lbfgs'])
    },
])

def f(params):
    with mlflow.start_run():
        classifier_type = params['type']
        params.pop('type', None)

        mlflow.log_params(params)
        mlflow.log_param('data', 'data/customers.csv')
        mlflow.set_tag('model', classifier_type)

        if classifier_type == 'svm':
            clf = SVC(**params)
        elif classifier_type == 'rf':
            clf = RandomForestClassifier(**params)
        elif classifier_type == 'logreg':
            clf = LogisticRegression(**params)
        else:
            return 0
        accuracy = cross_val_score(clf, X, y).mean()
        
        mlflow.log_metric('accuracy', accuracy)
    
        return {'loss': -accuracy, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, search_space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

  5%|██▍                                              | 5/100 [00:03<00:59,  1.58trial/s, best loss: -0.8749835530323334]

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

 30%|██████████████▍                                 | 30/100 [00:13<00:06, 10.02trial/s, best loss: -0.8756804171438318]

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

 34%|████████████████▎                               | 34/100 [00:15<00:13,  4.75trial/s, best loss: -0.8756804171438318]

...

 85%|█████████████████████████████████████████▋       | 85/100 [00:22<00:01,  9.48trial/s, best loss: -0.882646621671012]

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

 96%|██████████████████████████████████████████████  | 96/100 [00:24<00:00,  8.89trial/s, best loss: -0.8833459223703126]

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

/opt/homebrew/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.o

100%|███████████████████████████████████████████████| 100/100 [00:24<00:00,  4.06trial/s, best loss: -0.8833459223703126]
{'SVM_C': 0.003976491562090982, 'classifier_type': 0, 'kernel': 1}


In [ ]:
##Save best model

In [ ]:
#best params taken from the MLflow interface
best_params = {
    'C': 0.4168082195454126,
    'kernel': 'rbf'
}


best_model = SVC(**best_params)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)
best_model.fit(X_train, y_train)

In [ ]:
import pickle

In [ ]:
with open('models/best_model.bin', 'wb') as f_out:
    pickle.dump(best_model, f_out)

In [ ]:
print("accuracy:", cross_val_score(best_model, X, y).mean())